# WEATHER CHAT BOT

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from itertools import chain
from nltk import NaiveBayesClassifier as nbc
import codecs
import json
from urllib.request import urlopen
import datefinder
import datetime,time
from dateutil import parser
from nltk.tag.stanford import StanfordNERTagger
import pywapi
import random
import string

# Get date

In [2]:
# def get_day_of_week(question):

#     question = question.lower()
#     matches = list(datefinder.find_dates(question))
#     words_in_question = nltk.word_tokenize(question)
#     today = datetime.date.today()
    
#     day_of_week = {
        
#         "today": lambda: (today.strftime('%A')),
#         "day after tomorrow": lambda: ((today + datetime.timedelta(days = 2)).strftime('%A')),
#         "tomorrow": lambda: ((today + datetime.timedelta(days = 1)).strftime('%A')),
#         "default": lambda: (today.strftime('%A'))
#     }
    
#     if len(matches) > 0:
#         date = matches[0]
#         day = date.strftime('%A')
    
#     else:
#         intersected_days = (words_in_question & day_of_week.keys())
#         day = [day_of_week[day]() for day in intersected_days][0] if intersected_days else day_of_week['default']()
#     return day

In [3]:
def get_day_of_week(question):

    question = question.lower()
    matches = list(datefinder.find_dates(question))

    if len(matches) > 0:
        date = matches[0]
        day = date.strftime('%A')
        
    else:
        if 'today' in question:
            day = ((datetime.date.today()).strftime('%A'))
            
        elif 'day after tomorrow' in question:
            day = ((datetime.date.today() + datetime.timedelta(days = 2)).strftime('%A'))
            
        elif 'tomorrow' in question:
            day = ((datetime.date.today() + datetime.timedelta(days = 1)).strftime('%A'))
            
        else:
            day = ((datetime.date.today()).strftime('%A'))
            
    return day

# Get Location

In [4]:
def get_location(question):
    
    location_tagger = StanfordNERTagger('Tagger/stanford-ner-2016-10-31/classifiers/english.all.3class.distsim.crf.ser.gz', 'Tagger/stanford-ner-2016-10-31/stanford-ner-3.7.0.jar')
    question = question.title()
    tag = location_tagger.tag(question.split())
    loc_word = ''
    for word,tag in tag:
        if(tag == 'LOCATION'):
            loc_word = loc_word + ", " + word
        loc_word = loc_word.strip()
    if loc_word == '':
        loc_word = 'No Location'
    return loc_word

# Get location id for that city

In [7]:
def get_location_id(city):
 # Get location ID for that city
    count = 0
    lookup = pywapi.get_location_ids(city)
    while len(lookup) != 1:
        if len(lookup) > 1:
            for key,value in lookup.items():
                if 'India' in value:
                    location_id = key
                    city = value
                    flag = False
                    break
        if flag == False:
            break
        if len(lookup) == 0:
            print('Curent Location ',city,' is unable to locate')
            city = input('Please provide the location correctly: ')
            count += 1
            lookup = pywapi.get_location_ids(city)
            if count == 2:
                city = input('Please provide the best nearest location, we are unable to find that for you : ')
                lookup = pywapi.get_location_ids(city)
    for k in lookup:
        location_id = k
    return location_id,city

# Get weather of that Day

In [5]:
def get_weather_of_day(weather_com,day_of_week):
    weekday = ''
    for get_day in range(len(weather_com['forecasts'])):
        if day_of_week == weather_com['forecasts'][get_day]['day_of_week']:
            weekday = get_day
            break
    return weekday

# Get weather for that location

In [9]:
def get_weather_com(location_id):
    weather_com = pywapi.get_weather_from_weather_com(location_id) 
    return weather_com

# Getting the train Data

In [10]:
def get_traindata():
    train_csv = pd.read_csv('weathertrain.csv',header= None,names = ['sentence','label'])
    subset = train_csv[['sentence', 'label']]
    tuples = [tuple(x) for x in subset.values]
    return tuples

# Training the data

In [11]:
def trainNBC():
    data = get_traindata()
    stop_words = set(stopwords.words("english"))
    d1 = [(' '.join(list((i for i in word_tokenize(sentence) if i not in stop_words))),tag) for sentence, tag in data ]
    vocabulary = set(chain(*[word_tokenize(i[0].lower()) for i in d1 if i not in stop_words]))
    feature_set = [({i:(i in word_tokenize(sentence.lower())) for i in vocabulary},tag) for sentence, tag in d1]
    classifier = nbc.train(feature_set)
    return vocabulary,classifier

# Classifier for getting class

In [12]:
def get_class(query):
    test_sentence = query
    vocabulary,classifier = trainNBC()
    featurized_test_sentence =  {i:(i in word_tokenize(test_sentence.lower())) for i in vocabulary}
    ans = classifier.classify(featurized_test_sentence)
    return ans

# Get Results

In [8]:
def get_weather(city,day_of_week,old_category):
  # Getting location id for that city  
    location_id, city = get_location_id(city)
 # Getting weather data for that city
    weather_com = pywapi.get_weather_from_weather_com(location_id)    
  # Getting weather data for that day
    weekday = get_weather_of_day(weather_com,day_of_week)
    if weekday == '':
        return None,None,None
    weekday_weather = weather_com['forecasts'][weekday]
    max_list = ['max','maximum','hot','hottest','most']
    min_list = ['min','minimum','least','coolest','coldest']
    avg_list = ['average','avg','mean']
    category = get_class(query)
    if category == None:
        category = old_category
    if category == 'temperature':
        output = ''
        flag = False
        for k in max_list:
            if k in query.lower():
                output = weekday_weather['high']
                flag = True
                break
        for k in min_list:
            if k in query.lower():
                output = weekday_weather['low']
                flag = True
                break
        for k in avg_list:
            if k in query.lower():
                output = (int(weekday_weather['high']) + int(weekday_weather['low']))/2
                flag = True
                break
        if flag == False:
            output = weather_com['current_conditions']['temperature']
        output = str(output) + ' degree celsius'
    elif category == 'rain':
        day_data = weekday_weather['day']['chance_precip']
        night_data = weekday_weather['night']['chance_precip']
        if day_data != '':
            day_data = 'Durind day -' + day_data + '%'
        if night_data != '':
            night_data = ' During night -' + night_data + '%'
        output = day_data + night_data
    elif category == 'weather':
        day_data = weekday_weather['day']['text']
        night_data = weekday_weather['night']['text']
        if day_data != '':
            day_data = 'During day - ' + day_data
        if night_data != '':
            night_data = ' During night -' + night_data
        output = day_data + night_data 
    return output,city,category

# Check for greetings 


In [13]:
GREETING_KEYWORDS = ("hello", "hi", "greetings", "sup", "what's up",)

GREETING_RESPONSES = ["'sup bro", "hey", "*nods*", "hey you get my snap?"]

def check_for_greeting(sentence):
    """If any of the words in the user's input was a greeting, return a greeting response"""
    for word in word_tokenize(sentence):
        if word.lower() in GREETING_KEYWORDS:
            return random.choice(GREETING_RESPONSES)
        else:
            return "Nothing"

# Get User Input and produce output

In [15]:
flag = True
count = 0
while(flag):
    old_category = None
    query = input('Hello! What do you want to know ? ')
    greet = check_for_greeting(query)
    while greet != 'Nothing':
        new_query = greet + ' How May I assist you ?'
        query = input(new_query)
        greet = check_for_greeting(query)
    question = "".join(l for l in query if l not in string.punctuation)
    city = get_location(question)
    if city == 'No Location' and count == 0:
        city = input('Please provide your location')
    elif city == 'No Location' and count != 0:
        city = old_city
        category = old_category
    else:
        count = 0
        print('Getting details for:',city[2:])
    day_of_week = get_day_of_week(question)
    weather,city,category = get_weather(city,day_of_week,old_category)
    if weather == None:
        print('Cannnot predict for data more than 5 days')
        exit
    print(weather)
    old_city= city
    old_category = category
    count += 1

Hello! What do you want to know ? temperature in pune?
Getting details for: Pune
32 degree celsius
Hello! What do you want to know ? weather in pune?
Getting details for: Pune
 During night -T-Storms
Hello! What do you want to know ? rain?
Durind day -0% During night -0%
Hello! What do you want to know ? minimum temperature?
23 degree celsius


KeyboardInterrupt: 